## Import Libs

In [1]:
import pandas as pd
import time

## Classes and Functions

In [2]:
class restricoes:
    
    def jornada_trab_violada(tempo_servico, menor_dist, jornada_trab_restante, df_distancia, prox_cliente):

        tempo_volta_deposito = salva_tempo_volta_deposito(df_distancia, prox_cliente)

        if ((tempo_servico + menor_dist + tempo_volta_deposito) < jornada_trab_restante):
            return False
        else:
            return True 

    def capac_veic_violada(demanda, capac_veiculo):

        if (demanda > capac_veiculo):
            return True
        else:
            return False
        
    def testa_veic_disp(v, capac_veiculo):
    
        qtd_veiculos = len(capac_veiculo)

        if (v+1 < qtd_veiculos):
            return True
        else:
            return False

class criacao_estrutura_dados:
    
    def cria_dfs(file_names):

        df_distancia = pd.read_csv(f"./{file_names[0]}")
        df_demanda_ts = pd.read_csv(f"./{file_names[1]}")
        df_veic_cap = pd.read_csv(f"./{file_names[2]}")

        return df_distancia, df_demanda_ts, df_veic_cap

    def salva_capacidade_veiculos(df_veic_cap):

        capac = []

        for veiculo in df_veic_cap['Veiculo']:
            cap = int(df_veic_cap.query("Veiculo == '{}'".format(veiculo))['Capacidade'].unique())
            capac.append(cap)

        return capac

    def salva_dados_dict(df_distancia, df_demanda_ts):

        dict_dados = {}
        valor_distancia = []
        col_busca_dados = 'D1'
        lista_dados_dict = [] #ordem dos dados: [demanda, tempo_servico]

        for val in df_distancia[col_busca_dados]:
            valor_distancia.append(val)

        i = 1
        for distancia in valor_distancia[1:]:
            cliente = df_distancia['Unnamed: 0'][i]
            demanda = int(df_demanda_ts.query("NOH == '{}'".format(cliente))['DEMANDA'].unique())
            tempo_servico = float(df_demanda_ts.query("NOH == '{}'".format(cliente))['TEMPO_SERVICO'].unique())
            lista_dados_dict = [demanda, tempo_servico]
            dict_dados[cliente] = lista_dados_dict
            i += 1

        return dict_dados
    #daqui pra cima eu peguei os dados imutáveis para cada instância de análise!

    
def salva_prox_cliente(df_distancia, col_busca_dados, clientes_testados, clientes_rot): #deverá receber o índice da coluna a ser varrida

    global col_busca_dist

    dict_cliente_dist = {}
    valor_distancia = []
    menor_dist = 999.0

    for val in df_distancia[col_busca_dados]:
        valor_distancia.append(val)
    
    i = 1
    for distancia in valor_distancia[1:]: #aqui usa 1 porque tem apenas 1 depósito. caso houvessem 2, seria 2 e
                                            #assim por diante

        chave_dist = distancia
        cliente = df_distancia['Unnamed: 0'][i]

        if cliente in clientes_testados or cliente in clientes_rot:
            i += 1
            continue

        dict_cliente_dist[chave_dist] = cliente
        i += 1

        if distancia < menor_dist:
            menor_dist = distancia
    
    prox_cliente = dict_cliente_dist.get(menor_dist)
    col_busca_dist = prox_cliente

    return prox_cliente, menor_dist

def loop_roteamento(df_distancia, capac_veiculo, v):
    
    global col_busca_dist
    col_busca_dist = 'D1'
    clientes_testados = []
    clientes_rot = []
    total_clientes = int(df_distancia.count().unique()) - 1
    dict_dem_ts = criacao_estrutura_dados.salva_dados_dict(df_distancia, df_demanda_ts)
    jornada_trab_restante = 483 #valor da jornada em minutos
        
    while (len(clientes_rot) + len(clientes_testados) != total_clientes):
                
        prox_cliente, menor_dist = salva_prox_cliente(df_distancia, col_busca_dist, clientes_testados, clientes_rot)
        #print("Prox a ser avaliado: {}".format(prox_cliente))
        demanda = dict_dem_ts.get(prox_cliente)[0]
        tempo_servico = dict_dem_ts.get(prox_cliente)[1]
        
        jornada_violada = restricoes.jornada_trab_violada(tempo_servico, menor_dist, jornada_trab_restante, df_distancia, prox_cliente)
        capac_violada = restricoes.capac_veic_violada(demanda, capac_veiculo[v])

        if (jornada_violada or capac_violada):
            
            tempo_volta_dep = 0
            clientes_testados.append(prox_cliente)
            #print("Testados: {}".format(clientes_testados))
            if (clientes_rot == []):
                col_busca_dist = 'D1'
            else:
                col_busca_dist = clientes_rot[-1] #isso garante que a coluna de busca será sempre a do último cliente roteado
            
            
        else:

            clientes_rot.append(prox_cliente)
            #print("Roteados: {}".format(clientes_rot))
            jornada_trab_restante -= (tempo_servico + menor_dist)
            capac_veiculo[v] -= demanda
            tempo_volta_dep = salva_tempo_volta_deposito(df_distancia, prox_cliente)
    
    return clientes_rot, clientes_testados, jornada_trab_restante, tempo_volta_dep


def arruma_df_distancia(clientes_rot):
        
    global df_dist_aux
    rows = list(df_dist_aux['Unnamed: 0'])
    index_list = []

    for elem in clientes_rot:
        indx = rows.index(elem)
        index_list.append(indx)

    df_dist_aux.drop(clientes_rot, axis=1, inplace=True) #deleta as colunas (axis = 1)
    df_dist_aux.drop(index_list, axis=0, inplace=True)   #deleta as linhas (axis = 0)
    df_dist_aux.reset_index(drop=True, inplace=True)

def salva_tempo_volta_deposito(df_distancia, prox_cliente):
    
    indx = list(df_distancia.columns).index(prox_cliente)
    return df_distancia.iloc[0][indx]

def salva_nome_instancia(file_names):
    
    name_with_csv = file_names[0].split("_")[3]
    instancia = name_with_csv.split(".")[0]
    return instancia

## Coding

In [3]:
start_time = time.time()
file_names_to_split = [['Distancia_Malha_Viaria_MB01.csv', 'Demanda_TempoServico_MB01.csv', 'Veiculos_Capacidades_MB01.csv'],
                      ['Distancia_Malha_Viaria_MB02.csv', 'Demanda_TempoServico_MB02.csv', 'Veiculos_Capacidades_MB02.csv'],
                      ['Distancia_Malha_Viaria_MB03.csv', 'Demanda_TempoServico_MB03.csv', 'Veiculos_Capacidades_MB03.csv'],
                      ['Distancia_Malha_Viaria_M0S1.csv', 'Demanda_TempoServico_M0S1.csv', 'Veiculos_Capacidades_M0S1.csv'],
                      ['Distancia_Malha_Viaria_M0S2.csv', 'Demanda_TempoServico_M0S2.csv', 'Veiculos_Capacidades_M0S2.csv'],
                      ['Distancia_Malha_Viaria_MP01.csv', 'Demanda_TempoServico_MP01.csv', 'Veiculos_Capacidades_MP01.csv'],
                      ['Distancia_Malha_Viaria_MP02.csv', 'Demanda_TempoServico_MP02.csv', 'Veiculos_Capacidades_MP02.csv'],
                      ['Distancia_Malha_Viaria_MP03.csv', 'Demanda_TempoServico_MP03.csv', 'Veiculos_Capacidades_MP03.csv']]


#loop principal
for file_names in file_names_to_split:

    instancia = salva_nome_instancia(file_names)
    dados_roteamento = []
    col_busca_dist = 'D1' #constante, todos os dataframes terão apenas 1 depósito e ele será nomeado como D1
    df_distancia, df_demanda_ts, df_veic_cap = criacao_estrutura_dados.cria_dfs(file_names)
    df_dist_aux = df_distancia.copy()
    capac_veiculo = criacao_estrutura_dados.salva_capacidade_veiculos(df_veic_cap)
    v = 0 #o index dos veículos começa sempre em 0 e vai até (len(veiculos) - 1)

    clientes_rot, clientes_test, jornada_trab_restante, tempo_volta_dep = loop_roteamento(df_distancia, capac_veiculo, v) #esse loop varre todos os clientes e salva nas
                                                                    #listas de roteados e testados

    #clientes_rot_inst = [instancia + "_" + "V" + str(v+1) + "_" + elem for elem in clientes_rot] #esse aqui pode ser útil se for
    #necessário salvar o veículo também

    clientes_rot_inst = [instancia + "_" + elem for elem in clientes_rot]
    clientes_nao_rot_inst = [instancia + "_" + elem for elem in clientes_test if clientes_test != []]
    #if (clientes_test != []):
    #    clientes_nao_rot_inst = [instancia + "_" + elem for elem in clientes_test]
    #else:
    #    clientes_nao_rot_inst = []
    lista_dados_rot = [clientes_rot_inst, clientes_nao_rot_inst, jornada_trab_restante-tempo_volta_dep, capac_veiculo[v]]
    dados_roteamento.append(lista_dados_rot)

    while (clientes_test != []):

        ha_veic_disp = restricoes.testa_veic_disp(v, capac_veiculo)

        if (ha_veic_disp):

            arruma_df_distancia(clientes_rot)
            v += 1
            clientes_rot, clientes_test, jornada_trab_restante, tempo_volta_dep = loop_roteamento(df_dist_aux, capac_veiculo, v)
            #clientes_rot_inst = [instancia + "_" + "V" + str(v+1) + "_" + elem for elem in clientes_rot] #esse aqui pode ser útil se for
            #necessário salvar o veículo também
            clientes_rot_inst = [instancia + "_" + elem for elem in clientes_rot]
            clientes_nao_rot_inst = [instancia + "_" + elem for elem in clientes_test if clientes_test != []]
            lista_dados_rot = [clientes_rot_inst, clientes_nao_rot_inst, jornada_trab_restante-tempo_volta_dep, capac_veiculo[v]]
            dados_roteamento.append(lista_dados_rot)

        else:

            break

    print("\n\n-->Instância analisada: {}".format(instancia))
    for index, elem in enumerate(dados_roteamento):

        print("Veículo: {}\nRoteados: {}\nNão Roteados: {}\nJornada de Trabalho Restante: {:.2f} minutos\nCapacidade Restante: {} kg\n".format(index+1, elem[0],
                                                                                               elem[1], elem[2], elem[3]))

    #if (clientes_test != []):

     #   print("Não há veículo disponível")
      #  print("Clientes não roteados: {}\n".format(clientes_test))


print("O algoritmo demorou {:.2f} segundos para gerar a resposta.\n\n".format(time.time() - start_time))



-->Instância analisada: MB01
Veículo: 1
Roteados: ['MB01_C7', 'MB01_C16', 'MB01_C10', 'MB01_C1', 'MB01_C4', 'MB01_C11', 'MB01_C3', 'MB01_C17', 'MB01_C18', 'MB01_C9', 'MB01_C13', 'MB01_C15', 'MB01_C14', 'MB01_C8', 'MB01_C2', 'MB01_C12', 'MB01_C20', 'MB01_C19']
Não Roteados: ['MB01_C6', 'MB01_C5', 'MB01_C34', 'MB01_C25', 'MB01_C32', 'MB01_C37', 'MB01_C23', 'MB01_C24', 'MB01_C29', 'MB01_C28', 'MB01_C21', 'MB01_C30', 'MB01_C33', 'MB01_C22', 'MB01_C27', 'MB01_C35', 'MB01_C26', 'MB01_C36', 'MB01_C31']
Jornada de Trabalho Restante: 12.55 minutos
Capacidade Restante: 53 kg

Veículo: 2
Roteados: ['MB01_C5', 'MB01_C6', 'MB01_C34', 'MB01_C25', 'MB01_C37', 'MB01_C32', 'MB01_C23', 'MB01_C24', 'MB01_C29', 'MB01_C21', 'MB01_C30', 'MB01_C22', 'MB01_C33', 'MB01_C28', 'MB01_C27', 'MB01_C35', 'MB01_C26', 'MB01_C36', 'MB01_C31']
Não Roteados: []
Jornada de Trabalho Restante: 97.34 minutos
Capacidade Restante: 81 kg



-->Instância analisada: MB02
Veículo: 1
Roteados: ['MB02_C32', 'MB02_C29', 'MB02_C30',



-->Instância analisada: MP03
Veículo: 1
Roteados: ['MP03_C73', 'MP03_C63', 'MP03_C70', 'MP03_C80', 'MP03_C84', 'MP03_C87', 'MP03_C79', 'MP03_C77', 'MP03_C62', 'MP03_C81', 'MP03_C31', 'MP03_C34', 'MP03_C32', 'MP03_C76', 'MP03_C37', 'MP03_C83', 'MP03_C44', 'MP03_C28', 'MP03_C45', 'MP03_C47', 'MP03_C61', 'MP03_C57', 'MP03_C41', 'MP03_C27', 'MP03_C55', 'MP03_C56', 'MP03_C30', 'MP03_C33', 'MP03_C29', 'MP03_C52', 'MP03_C40', 'MP03_C53', 'MP03_C48', 'MP03_C58', 'MP03_C50', 'MP03_C54', 'MP03_C38', 'MP03_C39', 'MP03_C36', 'MP03_C42', 'MP03_C72']
Não Roteados: ['MP03_C46', 'MP03_C60', 'MP03_C35', 'MP03_C49', 'MP03_C86', 'MP03_C69', 'MP03_C66', 'MP03_C85', 'MP03_C78', 'MP03_C64', 'MP03_C75', 'MP03_C65', 'MP03_C74', 'MP03_C26', 'MP03_C12', 'MP03_C4', 'MP03_C68', 'MP03_C67', 'MP03_C82', 'MP03_C71', 'MP03_C43', 'MP03_C20', 'MP03_C15', 'MP03_C2', 'MP03_C9', 'MP03_C6', 'MP03_C59', 'MP03_C25', 'MP03_C51', 'MP03_C19', 'MP03_C22', 'MP03_C11', 'MP03_C14', 'MP03_C5', 'MP03_C17', 'MP03_C3', 'MP03_C16', 'M